In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

from torch.distributions.log_normal import LogNormal

# Internal packages
from data_loader import KidneyStoneDataset, ToTensor
from model import cont_size_ks_net, cont_rec_ks_net, binary_ks_net, binary_neg_loglik, cont_rec_neg_loglik, cont_size_neg_loglik
from train import train

In [2]:
# Hyperparameters
BATCH_SIZE = 128
EPOCHS     = 150
LEARN_R    = 1e-2
N_HU       = 3

# Initialize the dataset
data = KidneyStoneDataset("./data/ks_cont_size_data.npy", transform=ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE)

# Initialize the model
model = cont_size_ks_net(N_HU)

# Optimizers
#optimizer = optim.SGD(model.parameters(), lr=LEARN_R, weight_decay=0.1)
optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R)

In [3]:
cum_loss = train(model, optimizer, cont_size_neg_loglik, train_loader, EPOCHS)

In [4]:
cum_loss

[array(14.059219, dtype=float32),
 array(10.680594, dtype=float32),
 array(9.782367, dtype=float32),
 array(9.098038, dtype=float32),
 array(8.780098, dtype=float32),
 array(8.961344, dtype=float32),
 array(8.5975065, dtype=float32),
 array(8.041711, dtype=float32),
 array(8.266239, dtype=float32),
 array(8.257077, dtype=float32),
 array(8.139321, dtype=float32),
 array(7.7476993, dtype=float32),
 array(7.5767727, dtype=float32),
 array(7.820023, dtype=float32),
 array(7.779661, dtype=float32),
 array(7.637472, dtype=float32),
 array(7.368798, dtype=float32),
 array(7.5941887, dtype=float32),
 array(7.131846, dtype=float32),
 array(7.2386208, dtype=float32),
 array(7.146355, dtype=float32),
 array(7.0819597, dtype=float32),
 array(7.1951036, dtype=float32),
 array(6.907487, dtype=float32),
 array(6.9585795, dtype=float32),
 array(6.858229, dtype=float32),
 array(6.9214134, dtype=float32),
 array(6.8275948, dtype=float32),
 array(6.890427, dtype=float32),
 array(6.839052, dtype=float32)

In [117]:
# First, we get the parameters of the size variable:
n_samples = 1
arbitrary_query = torch.tensor([1., 1., 1.]) # It is only important that the first element is 1
mu_L, log_sigma_L, _, _, _ = model(arbitrary_query.unsqueeze(0))
sigma_L = torch.exp(log_sigma_L)
L_dist = LogNormal(mu_L, sigma_L)
L_samples = L_dist.sample((n_samples,)).view(n_samples,1)
L_prob = torch.exp(L_dist.log_prob(L_samples))

In [118]:
t_1 = torch.cat((L_samples, torch.ones(n_samples, 2)), 1)
t_0 = torch.cat((L_samples, torch.zeros(n_samples, 1) ,torch.ones(n_samples, 1)), 1)

In [119]:
# Probabilities with Treatment A
_, _, _, mu_t1, log_sigma_t1 = model(t_1) # Probabilities with KS = L
_, _, _, mu_t0, log_sigma_t0 = model(t_0) # Probabilities with KS = L

In [120]:
# By monte carlo integration, we are getting mu from the value of the variables rom the distribution
torch.mean(mu_t1) - torch.mean(mu_t0) # lognormal 1 sample 4.1766, 5 samples 4.1110, 20 samples 4.0318, 50 samples 4.1502, 200 samples 4.1494, 2k samples 4.1117

tensor(4.1766, grad_fn=<SubBackward0>)

In [123]:
model.__class__.__name__

'cont_size_ks_net'